In [2]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from PIL import Image

def csv_to_image(csv_file, image_file, image_size=(512, 512)):
    polylines = {}
    
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            try:
                polyline_id = int(float(row[0]))  # Convert to float first, then to int
                num_points = int(float(row[1]))  # Convert to float first, then to int
                
                if num_points <= 0:
                    print(f"Invalid number of points ({num_points}) for polyline_id {polyline_id}. Skipping.")
                    continue
                
                x_coords = list(map(float, row[2:2+num_points]))
                y_coords = list(map(float, row[2+num_points:2+2*num_points]))

                if len(x_coords) != num_points or len(y_coords) != num_points:
                    print(f"Inconsistent number of coordinates for polyline_id {polyline_id}. Skipping.")
                    continue

                if polyline_id not in polylines:
                    polylines[polyline_id] = (x_coords, y_coords)
                    
            except ValueError as e:
                print(f"Error parsing row {row}: {e}")
                continue

    # Create the image
    fig, ax = plt.subplots(figsize=(image_size[0] / 100, image_size[1] / 100), dpi=100)
    
    for polyline_id, (x_coords, y_coords) in polylines.items():
        if len(x_coords) < 2 or len(y_coords) < 2:
            print(f"Skipping polyline_id {polyline_id} due to insufficient points ({len(x_coords)})")
            continue

        try:
            polygon = Polygon(list(zip(x_coords, y_coords)), closed=False, edgecolor='black')
            ax.add_patch(polygon)
        except Exception as e:
            print(f"Error creating polygon for polyline_id {polyline_id}: {e}")
            continue

    ax.set_xlim(0, image_size[0])
    ax.set_ylim(0, image_size[1])
    ax.set_aspect('equal')
    plt.gca().invert_yaxis()
    plt.axis('off')
    plt.savefig(image_file, bbox_inches='tight', pad_inches=0)
    plt.close()

# Define paths
input_csv_folder = 'input_csv'
output_csv_folder = 'output_csv'
input_image_folder = 'input_images'
output_image_folder = 'output_images'

# Create directories if they don't exist
os.makedirs(input_image_folder, exist_ok=True)
os.makedirs(output_image_folder, exist_ok=True)

# Convert CSVs in input_csv_folder to PNG images in input_image_folder
for csv_file in os.listdir(input_csv_folder):
    if csv_file.endswith('.csv'):
        csv_path = os.path.join(input_csv_folder, csv_file)
        image_path = os.path.join(input_image_folder, csv_file.replace('.csv', '.png'))
        csv_to_image(csv_path, image_path)

# Convert CSVs in output_csv_folder to PNG images in output_image_folder
for csv_file in os.listdir(output_csv_folder):
    if csv_file.endswith('.csv'):
        csv_path = os.path.join(output_csv_folder, csv_file)
        image_path = os.path.join(output_image_folder, csv_file.replace('.csv', '.png'))
        csv_to_image(csv_path, image_path)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'input_csv'

In [ ]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

def load_images_from_folder(folder, size=(512, 512)):
    images = []
    for filename in sorted(os.listdir(folder)):
        if filename.endswith('.png'):
            img_path = os.path.join(folder, filename)
            image = Image.open(img_path).resize(size).convert('RGB')
            images.append(np.array(image))
    return np.array(images)

# Define paths
input_image_folder = 'input_images'  # Folder containing your input PNG images
output_image_folder = 'output_images'  # Folder containing your output PNG images

# Load images
x_images = load_images_from_folder(input_image_folder)
y_images = load_images_from_folder(output_image_folder)

# Ensure same number of images and correct shapes
assert x_images.shape[0] == y_images.shape[0], "Number of input and output images must match"
assert x_images.shape[1:] == (512, 512, 3), "Image dimensions must match the specified size"

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_images, y_images, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam

def build_model():
    inputs = Input(shape=(512, 512, 3))
    
    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    # Bottleneck
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    
    # Decoder
    up5 = UpSampling2D(size=(2, 2))(conv4)
    concat5 = concatenate([up5, conv3], axis=-1)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(concat5)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)
    
    up6 = UpSampling2D(size=(2, 2))(conv5)
    concat6 = concatenate([up6, conv2], axis=-1)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)
    
    up7 = UpSampling2D(size=(2, 2))(conv6)
    concat7 = concatenate([up7, conv1], axis=-1)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)
    
    outputs = Conv2D(3, (1, 1), activation='sigmoid')(conv7)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build and train the model
model = build_model()
model.summary()

# Train the model
history = model.fit(x_train, y_train, epochs=50, batch_size=1, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# Predict on test images
predictions = model.predict(x_test)

# Save predictions as images
os.makedirs('predictions', exist_ok=True)
for i, pred in enumerate(predictions):
    pred_img = Image.fromarray((pred * 255).astype(np.uint8))
    pred_img.save(f'predictions/pred_{i}.png')


In [ ]:
import matplotlib.pyplot as plt

# Predict on test images
predictions = model.predict(x_test)

# Function to display images
def display_images(original, predicted, index):
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(original[index])
    axs[0].set_title('Original Image')
    axs[0].axis('off')
    
    axs[1].imshow(predicted[index])
    axs[1].set_title('Predicted Image')
    axs[1].axis('off')
    
    plt.show()

# Display the first test image and its prediction
display_images(x_test, predictions, 0)
